#### 스태킹(Stacking)

- 스태킹은 여러 모델의 예측 결과를 메타 모델(meta-model)로 결합하는 방법
- 각 기본 모델(base model)이 예측한 결과를 새로운 데이터로 사용하여 메타 모델이 학습됨
- 기본 모델과 메타 모델이 서로 다른 알고리즘을 사용할 수 있음

![](https://velog.velcdn.com/images/newnew_daddy/post/53adabda-652c-4291-bf35-8306340b18bb/image.png)

#### 배깅 vs 부스팅 vs 스태깅

![](https://velog.velcdn.com/images/newnew_daddy/post/02139a3a-06cd-4fd9-b00e-40e3988ded13/image.png)
